In [ ]:
# Install kaggle for datasets
!pip install kaggle --quiet

In [ ]:
#importing neccesary libraries

import sklearn
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

import random
import typing
import time
from pathlib import Path
from tqdm.notebook import tqdm, trange
import os
import re

In [ ]:
# download the 20-newsgroups dataset from kaggle
!kaggle datasets download -d crawford/20-newsgroups

Dataset URL: https://www.kaggle.com/datasets/crawford/20-newsgroups
License(s): other
 74% 19.0M/25.7M [00:00<00:00, 44.3MB/s]
100% 25.7M/25.7M [00:00<00:00, 47.0MB/s]


In [ ]:
# unzip the downloader file
import zipfile
zip_ref = zipfile.ZipFile('/content/20-newsgroups.zip', 'r')
zip_ref.extractall('20-newsgroups')

In [ ]:
# list of all the news group
news_groups = os.listdir('20-newsgroups')

In [ ]:
# there is a csv file we don't need that that's why remove that
del news_groups[2]

In [ ]:
def creat_documents(news_groups, path):
  """
  function for split every news group corpus into multiple chunks
  """
  documents = []
  for news_group in news_groups:
    with open(os.path.join(path, news_group), 'r', encoding='latin-1') as f:
      text = f.read()
      print('Newsgroup: ' + news_group[:-4])
    document_list = text.split('Newsgroup: ' + news_group[:-4])
    document_list = [document for document in document_list if len(document) < 5000 and len(document)>0]
    documents.extend(document_list)
  return documents

documents = creat_documents(news_groups, '/content/20-newsgroups')

Newsgroup: rec.sport.baseball
Newsgroup: alt.atheism
Newsgroup: sci.crypt
Newsgroup: talk.religion.misc
Newsgroup: talk.politics.misc
Newsgroup: comp.os.ms-windows.misc
Newsgroup: comp.sys.mac.hardware
Newsgroup: sci.electronics
Newsgroup: talk.politics.guns
Newsgroup: comp.graphics
Newsgroup: talk.politics.mideast
Newsgroup: comp.sys.ibm.pc.hardware
Newsgroup: rec.motorcycles
Newsgroup: rec.autos
Newsgroup: rec.sport.hockey
Newsgroup: sci.space
Newsgroup: comp.windows.x
Newsgroup: sci.med
Newsgroup: misc.forsale
Newsgroup: soc.religion.christian


In [ ]:
print(f'total number of documents is {len(documents)}')

total number of documents is 36018


In [ ]:
def clean_document(document):
    # Lowercase the document
    document = document.lower()

    # Remove email IDs
    document = re.sub(r'\S+@\S+', '', document)

    # Remove lines starting with specific headers
    document = re.sub(r'^(from|subject|summary|keywords|date|document_id|in article):\s*.*$', '', document, flags=re.MULTILINE)

    # Remove all punctuations
    document = re.sub(r'[^\w\s]', '', document)

    # Remove footers (lines with repeated underscores or patterns indicating footer)
    document = re.sub(r'_{2,}.*', '', document, flags=re.MULTILINE)

    # Remove multiple spaces and replace with single space
    document = re.sub(r'\s+', ' ', document)

    # Remove leading and trailing whitespace
    document = document.strip()

    return document

print(clean_document(documents[10]))

could somebody please tell me if there is a dodgers newsletter on the net and if so how to subscribe thanks joel


In [ ]:
documents =  [clean_document(document) for document in documents]

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopword = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
documents = [' '.join([word for word in document.split() if word not in stopword]) for document in documents]

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModel

model_name = "NeuML/pubmedbert-base-embeddings"

# Load pretrained BERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("NeuML/pubmedbert-base-embeddings")
model = AutoModel.from_pretrained("NeuML/pubmedbert-base-embeddings")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)

# Function to generate embeddings for a document
def get_document_embedding(document, tokenizer, model):
    # Tokenize the input document
    inputs = tokenizer(
        document,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512  # BERT has a max token limit of 512
    )

    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Pass through the BERT model
    with torch.no_grad():
        outputs = model(**inputs)

    # extract the word embedding
    token_embedding = outputs.last_hidden_state[0][1:-1]
    return tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][1:-1]), token_embedding.cpu().numpy()

In [ ]:
# document = ["hi i am good because i forget codings"]

# inputs = tokenizer(
#     document,
#     padding=True,
#     truncation=True,
#     max_length=512,
#     return_tensors="pt"
# )
# print(model(**inputs)[0].shape)
# print(inputs['input_ids'])
# tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

In [ ]:
x, y = get_document_embedding(documents[:3], tokenizer, model)

In [ ]:
word_embeddings = {}
documents_embeddings = []
for document in tqdm(documents):
  tokens, embedding = get_document_embedding(document, tokenizer, model)
  documents_embeddings.append([tokens, embedding])
  for token, emb in zip(tokens, embedding):
    word_embeddings[token] = emb

  0%|          | 0/36018 [00:00<?, ?it/s]

In [ ]:
# create dataframe of word and thier embedding
df = pd.DataFrame(word_embedding.values(), index=word_embedding.keys())

In [ ]:
df

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
article,0.675349,0.138285,0.382839,-0.479901,0.433749,-0.050931,0.179182,-0.537125,0.327359,-0.276972,...,0.593854,-0.216850,-0.180720,-0.173886,0.109595,-0.052334,-0.937258,0.497774,0.443980,-0.235572
writes,0.340408,1.082828,0.428304,0.074668,0.551918,-0.306200,0.099694,0.297168,0.329991,-0.278741,...,-0.313944,0.314692,-0.954563,0.147812,0.114701,0.115755,0.022013,-0.143487,-0.275800,0.119701
variety,0.045539,-0.312820,0.878454,-0.178505,0.348240,-0.324186,-0.018992,0.334038,-0.351069,-0.462366,...,-0.136152,-0.028113,0.686264,0.173662,-0.158837,-0.313863,0.155523,-0.300905,-0.075962,0.223289
water,0.039334,0.497111,0.026864,0.314274,0.602603,0.032274,0.170242,-0.308216,-0.301589,0.356624,...,-0.084154,-0.497405,0.206701,0.124750,0.343408,-0.003278,0.523661,-0.077643,-0.075717,0.149712
##proof,0.109190,0.783719,0.388885,0.121665,0.144103,-0.444140,-0.133922,0.028763,0.416585,-0.092839,...,-0.009067,0.217790,-0.414812,-0.035998,-0.073328,0.001406,-0.150803,0.093942,-0.125676,-0.362484
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
caretaker,-1.120447,0.554155,0.142890,-0.263975,-0.242854,-0.007587,0.127931,0.185707,0.212583,-0.515354,...,-0.033781,-0.472060,-0.113854,0.581675,0.231418,-0.183393,-0.041639,-0.589551,0.033717,0.408596
tits,0.943552,-0.069961,0.435097,-0.000724,0.239692,0.135994,0.272007,0.194860,-0.639481,-0.419192,...,-0.984207,-0.248082,-0.985961,0.226714,-1.042925,-0.110846,0.566111,-0.028224,-0.185801,-0.886171
##cellular,0.597162,0.820727,0.408312,0.038783,1.347533,-0.107692,-0.169068,0.414025,0.116455,-0.807731,...,-0.992511,0.056979,0.483682,0.848347,0.308963,0.118441,-0.773079,-0.285931,-0.209579,-0.271527
calming,-1.006219,-0.174232,0.931650,-0.289022,-0.236282,-0.310935,0.365506,0.536075,-0.855140,-0.362761,...,0.310711,-0.203264,0.282967,0.247910,0.366584,0.436439,-0.223622,-0.408274,-0.703018,-1.061681


In [ ]:
embedding_vectors = df.values

In [ ]:
embedding_vectors.shape

(24457, 768)

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
from sklearn.metrics import silhouette_score
def getbestk(data):
  inertia = []
  silhouette = []
  for i in range(10, 50, 10):
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(data)
    inertia.append(kmeans.inertia_)
    silhouette.append(silhouette_score(data, kmeans.labels_))
  return inertia, silhouette

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=500)
pca.fit(embedding_vectors)
pca.explained_variance_ratio_.sum()

0.94644046

In [ ]:
embedding_vector_compress = pca.transform(embedding_vectors)

In [ ]:
inertia, silahoette = getbestk(embedding_vector_compress)

In [ ]:
inertia, silahoette

([2878587.75, 2802276.75, 2756294.5, 2726106.5],
 [-0.010366135, -0.044603504, -0.055422217, -0.05983475])